# Model Development

Begin documenting models

In [17]:
import pandas as pd
import requests
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import string, re
# import pandas_profiling
import itertools 
import pickle
import local_modules.slack as slack
from local_modules.Pickling import pickle_item

In [18]:
%load_ext autoreload

In [19]:
%autoreload 2

In [3]:
test_features = pickle.load(open('test_features.pkl', 'rb'))['test_features']
test_labels = pickle.load(open('test_labels.pkl', 'rb'))['test_labels']
train_features = pickle.load(open('train_features.pkl', 'rb'))['train_features']
train_labels = pickle.load(open('train_labels.pkl', 'rb'))['train_labels']

In [7]:
loaded_model = pickle.load(open('voting_classifier.pkl', 'rb'))
loaded_model

{'eclf1': VotingClassifier(estimators=[('lr',
                               LogisticRegression(C=1.0, class_weight=None,
                                                  dual=False, fit_intercept=True,
                                                  intercept_scaling=1,
                                                  l1_ratio=None, max_iter=100,
                                                  multi_class='multinomial',
                                                  n_jobs=None, penalty='l2',
                                                  random_state=1, solver='lbfgs',
                                                  tol=0.0001, verbose=0,
                                                  warm_start=False)),
                              ('rf',
                               RandomForestClassifier(bootstrap=True,
                                                      class_weight=None,
                                                      criterion='g...
                 

In [11]:
print(train_labels.shape)
print(train_features.shape)
print(test_labels.shape)
print(test_features.shape)

(11666,)
(11666, 3111)
(3889,)
(3889, 3111)


## Sample model #1: Random Forest

Testing a random forest model on predicting fake news

In [5]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
classifier = RandomForestClassifier(n_estimators = 1000, criterion = 'entropy', random_state = 0)
classifier.fit(train_features, train_labels)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [6]:
filehandler = open("models/RandomForestClassifier.pkl","wb")
pickle.dump(classifier, filehandler)
filehandler.close()

In [12]:
# Import the model we are using
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, VotingClassifier

clf1 = LogisticRegression(solver='lbfgs', multi_class='multinomial',
                          random_state=1)
clf2 = RandomForestClassifier(n_estimators=50, random_state=1)
clf3 = GaussianNB()

eclf1 = VotingClassifier(estimators=[
        ('lr', clf1), ('rf', clf2), ('gnb', clf3)], voting='hard')
eclf1 = eclf1.fit(train_features, train_labels)
print(eclf1.predict(train_features))

[0 0 1 ... 1 1 0]


In [16]:
print(eclf1.predict(train_features)[1:10])
print(test_labels[1:10])

[0 1 0 2 0 0 0 2 0]
12766    2
6461     0
9896     0
13496    0
5099     1
9394     1
1688     1
8447     0
720      1
Name: label, dtype: int64


In [21]:
pickle_item("models/test.pkl", eclf1)

Pickling completed


In [ ]:
X = np.array([[-1, -1], [-2, -1], [-3, -2], [1, 1], [2, 1], [3, 2]])
y = np.array([1, 1, 1, 2, 2, 2])



np.array_equal(eclf1.named_estimators_.lr.predict(X),
               eclf1.named_estimators_['lr'].predict(X))

eclf2 = VotingClassifier(estimators=[
        ('lr', clf1), ('rf', clf2), ('gnb', clf3)],
        voting='soft')
eclf2 = eclf2.fit(X, y)
print(eclf2.predict(X))

eclf3 = VotingClassifier(estimators=[
       ('lr', clf1), ('rf', clf2), ('gnb', clf3)],
       voting='soft', weights=[2,1,1],
       flatten_transform=True)
eclf3 = eclf3.fit(X, y)
print(eclf3.predict(X))

print(eclf3.transform(X).shape)


## Brute force models using Hunga Bunga

In [4]:
from local_modules.hunga_bunga import HungaBungaClassifier

In [ ]:
clf = HungaBungaClassifier(brain=True)
clf.fit(train_features, train_labels)

Scoring criteria: accuracy
--------------- model 1/15 ---------------
SGDClassifier
--------------- model 2/15 ---------------
LogisticRegression
--------------- model 3/15 ---------------
Perceptron
--------------- model 4/15 ---------------
PassiveAggressiveClassifier
--------------- model 5/15 ---------------
MLPClassifier
best score: 0.5143958868894601 time/clf: 2.819 seconds
best params:
{'activation': 'tanh',
 'batch_size': 50,
 'early_stopping': True,
 'hidden_layer_sizes': (64,),
 'learning_rate': 'invscaling',
 'max_iter': 500}
--------------- model 6/15 ---------------
KMeans
best score: 0.407369323050557 time/clf: 0.149 seconds
best params:
{'algorithm': 'elkan', 'init': 'random', 'n_clusters': 3}
--------------- model 7/15 ---------------
KNeighborsClassifier
--------------- model 8/15 ---------------
NearestCentroid
best score: 0.4011139674378749 time/clf: 0.004 seconds
best params:
{'metric': 'manhattan', 'shrink_threshold': 2}
--------------- model 9/15 ---------------
R

In [4]:
loaded_model = pickle.load(open("models/model.pkl", 'rb'))
result = loaded_model.score(X_test, Y_test)
print(result)

ModuleNotFoundError: No module named 'azureml'